# Metadata

```yaml
Course:   DS5001: Exploratory Text Analytics
Topic:    Final Project, Create Tables
Author:   Andrew Avitabile
Date:     24 March 2024 (Edited April 25, 2024)
```

# Set Up

## Packages

In [1]:
# Importing required libraries
import pandas as pd
import numpy as np
from collections import Counter
from numpy.linalg import norm

# sklearn
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline, AutoModelForSeq2SeqLM

In [2]:
# Define the base path
base_path = "C:/Users/Andre/Box/DS5001 Final Project/"

## Import Data

In [3]:
LIB = pd.read_csv(base_path + "output/LIB.csv", delimiter = "|").set_index('observationid')
CORPUS = pd.read_csv(base_path + "output/CORPUS.csv", delimiter = "|").set_index(['observationid', 'sentence_num', 'token_num'])
VOCAB = pd.read_csv(base_path + "output/VOCAB.csv", delimiter = "|").set_index('term_str')

In [4]:
# Replace NaN values with an empty string (or another placeholder)
CORPUS['term_str'] = CORPUS['term_str'].fillna('')

# Create tables

## Bag-of-words (BOW)

In [5]:
def create_bow(CORPUS, bag, item_type='term_str'):
    BOW = CORPUS.groupby(bag+[item_type])[item_type].count().to_frame('n')
    return BOW

In [6]:
BOW_document = create_bow(CORPUS, bag=['observationid'])
BOW_document

n
observationid term_str       
0             \n\n          4
                           18
              "             6
              %             1
              're           1
...                        ..
19007         questioning   1
              students      1
              technique     1
              the           1
              throughout    1

[1415978 rows x 1 columns]

In [7]:
BOW_document.to_csv(base_path + "output/BOW_document.csv", sep='|', index=True)

## Create Document-Term Count Matrix (DCTM), DFIDF, and TFIDF

In [8]:
def get_tfidf(BOW, tf_method='max', df_method='standard', item_type='term_str'):
            
    DTCM = BOW.n.unstack() # Create Doc-Term Count Matrix
    
    if tf_method == 'sum':
        TF = (DTCM.T / DTCM.T.sum()).T
    elif tf_method == 'max':
        TF = (DTCM.T / DTCM.T.max()).T
    elif tf_method == 'log':
        TF = (np.log2(DTCM.T + 1)).T
    elif tf_method == 'raw':
        TF = DTCM
    elif tf_method == 'bool':
        TF = DTCM.astype('bool').astype('int')
    else:
        raise ValueError(f"TF method {tf_method} not found.")

    DF = DTCM.count() # Assumes NULLs 
    N_docs = len(DTCM)
    
    if df_method == 'standard':
        IDF = np.log2(N_docs/DF) # This what the students were asked to use
    elif df_method == 'textbook':
        IDF = np.log2(N_docs/(DF + 1))
    elif df_method == 'sklearn':
        IDF = np.log2(N_docs/DF) + 1
    elif df_method == 'sklearn_smooth':
        IDF = np.log2((N_docs + 1)/(DF + 1)) + 1
    else:
        raise ValueError(f"DF method {df_method} not found.")
    
    TFIDF = TF * IDF
    
    DFIDF = DF * IDF
    
    TFIDF = TFIDF.fillna(0)
    
    DTCM = DTCM.fillna(0)

    return TFIDF, DFIDF, DTCM, TF, DF, IDF

In [9]:
TFIDF, DFIDF, DTCM, TF, DF, IDF = get_tfidf(BOW = BOW_document, tf_method = "max")

In [10]:
TFIDF.to_csv(base_path + "output/TFIDF.csv", sep='|', index=True)

In [11]:
DFIDF.to_csv(base_path + "output/DFIDF.csv", sep='|', index=True)

In [12]:
DTCM.to_csv(base_path + "output/DTCM.csv", sep='|', index=True)

In [13]:
VOCAB['df'] = DF
VOCAB['idf'] = IDF
VOCAB['dfidf'] = DFIDF
VOCAB['mean_tfidf'] = TFIDF.mean()

In [14]:
VOCAB.to_csv(base_path + "output/VOCAB.csv", sep='|', index=True)

In [15]:
# Sorting by DFIDF to find the top 20 significant words
top_20_significant = VOCAB.sort_values(by='dfidf', ascending=False).head(20)
top_20_significant

,n,porter_stem,stop,max_pos,max_pos_group,ngram_length,df,idf,dfidf,mean_tfidf
term_str,,,,,,,,,,
were,12611,were,True,VBD,VERB,1,6770.0,1.489379,10083.095846,0.109373
as,12849,as,True,IN,OTHER,1,6721.0,1.499859,10080.551884,0.106608
you,32847,you,True,PRP,OTHER,1,6681.0,1.508471,10078.093263,0.194298
-,12348,-,False,HYPH,OTHER,1,6425.0,1.564838,10054.086603,0.116227
student,12205,student,False,NN,NOUN,1,6412.0,1.567760,10052.479753,0.114377
are,11637,are,True,VBP,VERB,1,6276.0,1.598689,10033.375201,0.109792
that,15048,that,True,IN,OTHER,1,7815.0,1.282289,10021.088245,0.111759
well,9083,well,False,RB,ADVERB,1,6173.0,1.622563,10016.081675,0.103210
was,15365,wa,True,VBD,VERB,1,7936.0,1.260123,10000.334623,0.124090


## Create a reduced and Normalized TFIDF_L2

In [16]:
bag = 'observationid'
vocab_filter = 'dfidf'
n_terms = 1000
pos_list = ['NN', 'VB', 'JJ'] #Limit to nouns, verbs, and ajectives

In [18]:
VIDX = VOCAB.loc[VOCAB.max_pos.isin(pos_list)]\
    .sort_values(vocab_filter, ascending=False)\
    .head(n_terms).index

In [19]:
M = TFIDF[VIDX].fillna(0).groupby('observationid').mean() # MUST FILLNA

In [20]:
TFIDF_L2 = M.apply(lambda x: x / norm(x), 1) # Euclidean

In [21]:
TFIDF_L2

term_str,student,good,classroom,job,teacher,learning,be,class,work,time,...,chair,calculate,recognition,tie,range,deliberate,proficiency,trip,description,responsive
observationid,,,,,,,,,,,,,,,,,,,,,
0,0.103370,0.000000,0.000000,0.000000,0.0,0.000000,0.031059,0.000000,0.032507,0.068137,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.021601,0.000000,0.000000,0.023791,0.0,0.077862,0.051923,0.026430,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.083492,0.084207,0.000000,0.0,0.000000,0.000000,0.096372,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.044880,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.088653,0.093976,0.000000,0.000000,0.0,0.000000,0.159824,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19003,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.186954,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19004,0.146316,0.077551,0.078215,0.000000,0.0,0.087900,0.000000,0.000000,0.092023,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19005,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.379755,0.000000,0.136387,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
TFIDF_L2.to_csv(base_path + "output/TFIDF_L2.csv", sep='|', index=True)